In [2]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

Cloning into 'TACL2015'...
remote: Enumerating objects: 2294, done.
remote: Counting objects: 100% (2294/2294), done.
remote: Compressing objects: 100% (2234/2234), done.
remote: Total 2294 (delta 203), reused 2103 (delta 55), pack-reused 0
Receiving objects: 100% (2294/2294), 4.51 MiB | 3.62 MiB/s, done.
Resolving deltas: 100% (203/203), done.
Cloning into 'nlu-asdiv-dataset'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 30 (delta 6), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (30/30), 425.56 KiB | 1.67 MiB/s, done.
Cloning into 'grade-school-math'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 14), reused 30 (delta 11), pack-reused 0
Unpacking objects: 100% (36/36), 3.01 MiB | 4.94 MiB/s, done.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored
import wandb

In [2]:
import dataset_handler as dh
#import helper_func as hf
import loading_utils as lu
import testing_utils as tu

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

#model_name = "gpt-j"
model_name = "codegen"

#dataset_name = "gsm8k"

In [3]:
import exp_impl.func_def_eq as exp_impl
#import exp_impl.func_def_mine as exp_impl
#import exp_impl.simple_func_def as exp_impl

In [5]:
import importlib
importlib.reload(tu)

<module 'testing_utils' from '/home/PracticalWork2021/testing_utils.py'>

In [6]:
"""Load gsm8k"""

if model_name == "gpt-j":
    priming_text_path = (
        "data/priming_texts/gsm8k/gpt-j/gsm8k_fewer_alt.txt"  # for gpt-j
    )
    current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )
else:
    priming_text_path = "data/priming_texts/gsm8k/codegen/gsm8k_fewer_alt_codegen_func_eq.txt"  # for codegen
    #priming_text_path = "data/priming_texts/gsm8k/codegen/gsm8k_fewer_alt_codegen_func.txt"
    current_dataset = dh.init_dataset_from_name(
        "gsm8k",
        primingtext_path=priming_text_path,
        sample_func=exp_impl.sample_n_for_prompting,
        generate_prompt_func=exp_impl.generate_prompt,
    )


In [5]:
"""Load asdiv"""

if model_name == "gpt-j":
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix.txt" # for gpt-j
else:
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt" # for codegen

current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

In [7]:
tu.set_all_seeds()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(50)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

def exercise6():
    """Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days? Hint: use these equations eq1: 960/4=240 eq2: 240/3=80"""
80


In [7]:
with open("test_prompt.txt", "w") as f:
    f.write(current_dataset.generate_prompt(sample_q_list[0]))

In [8]:
if model_name == "gpt-j":
    """GPT-J and codeparrot models run in HFTest venv"""
    tokenizer = AutoTokenizer.from_pretrained(gptj_model)
    model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()
elif model_name == "codegen":
    """CodeGen runs in the venv venv"""
    model_args = lu.model_args()
    #model_args.model = "codegen-350M-mono"
    model, tokenizer = lu.load_CodeGen(model_args)

loading parameters
loading parameters took 696.62s
loading tokenizer
loading tokenizer took 4.02s


In [9]:
# Set up for CodeGen
config = lu.codegen_gen_args()
config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.k = 3
config.max_lenght_after_input = 250
#config.top_p = 0.95
config.top_p = 0.95
config.top_k = 50
#config.temperature = 0.7
config.temperature = 0.61
config.min_length = 3

tu.set_all_seeds(model_name)
tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

TESTING STARTED
@sample 5 -> Pass@3 = 0.5
@sample 10 -> Pass@3 = 0.425
@sample 15 -> Pass@3 = 0.2833333333333333
@sample 20 -> Pass@3 = 0.3
@sample 25 -> Pass@3 = 0.3
@sample 30 -> Pass@3 = 0.275
@sample 35 -> Pass@3 = 0.30714285714285716
@sample 40 -> Pass@3 = 0.3125
@sample 45 -> Pass@3 = 0.31666666666666665
@sample 50 -> Pass@3 = 0.32


Pass@3 = 0.32


0.32

In [ ]:
# Set up for gpt-j
config = lu.gptj_gen_args()

tu.set_all_seeds(model_name)
#hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, print_output=False)

In [24]:
with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name="@500-gsm8k-codegen-func-def-eq"):

        tu.set_all_seeds(model)
        pass_at_k = tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

        wandb.log({"pass_at_k": pass_at_k})

TESTING STARTED
@sample 5 -> Pass@3 = 0.45
@sample 10 -> Pass@3 = 0.4
@sample 15 -> Pass@3 = 0.26666666666666666
@sample 20 -> Pass@3 = 0.325
@sample 25 -> Pass@3 = 0.32
@sample 30 -> Pass@3 = 0.26666666666666666
@sample 35 -> Pass@3 = 0.32857142857142857
@sample 40 -> Pass@3 = 0.3125
@sample 45 -> Pass@3 = 0.31666666666666665
@sample 50 -> Pass@3 = 0.32
@sample 55 -> Pass@3 = 0.32272727272727275
@sample 60 -> Pass@3 = 0.325
@sample 65 -> Pass@3 = 0.3153846153846154
@sample 70 -> Pass@3 = 0.3142857142857143
@sample 75 -> Pass@3 = 0.33
@sample 80 -> Pass@3 = 0.328125
@sample 85 -> Pass@3 = 0.3264705882352941
@sample 90 -> Pass@3 = 0.30833333333333335
@sample 95 -> Pass@3 = 0.31842105263157894
@sample 100 -> Pass@3 = 0.3025
@sample 105 -> Pass@3 = 0.32857142857142857
@sample 110 -> Pass@3 = 0.3386363636363636
@sample 115 -> Pass@3 = 0.3239130434782609
@sample 120 -> Pass@3 = 0.31666666666666665
@sample 125 -> Pass@3 = 0.318
@sample 130 -> Pass@3 = 0.3211538461538462
@sample 135 -> Pass@3

pass_at_k,▁
pass_at_k,0.322
